In [49]:
# Import the hive library from pyhive
from pyhive import hive

In [50]:
#load the database from hive

In [117]:
conn=hive.Connection(host="localhost",port=10000,auth='NONE',username='suran',database="london_crimes")

In [55]:
#import pandas and load the hive conn which now contains the london crimes database into pandas data frame for analysis

In [56]:
import pandas as pd

In [57]:
df=pd.read_sql('SELECT * from londoncrimes limit 5000',conn)

In [58]:
#check the first 10 rows of data using head. Note only loaded 5000 records

In [59]:
df.head(10)

,londoncrimes.lsoa_code,londoncrimes.borough,londoncrimes.major_category,londoncrimes.minor_category,londoncrimes.value,londoncrimes.year
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008
5,E01001320,Ealing,Theft and Handling,Other Theft,0,2012
6,E01001342,Ealing,Violence Against the Person,Offensive Weapon,0,2010
7,E01002633,Hounslow,Robbery,Personal Property,0,2013
8,E01003496,Newham,Criminal Damage,Criminal Damage To Other Building,0,2013
9,E01004177,Sutton,Theft and Handling,Theft/Taking of Pedal Cycle,1,2016


In [60]:
df.columns

Index([u'londoncrimes.lsoa_code', u'londoncrimes.borough',
       u'londoncrimes.major_category', u'londoncrimes.minor_category',
       u'londoncrimes.value', u'londoncrimes.year'],
      dtype='object')

In [61]:
#Lets check the top 30 crimes by Borough 

In [62]:
df.groupby(['londoncrimes.borough','londoncrimes.major_category'])['londoncrimes.value'].sum().head(30)

londoncrimes.borough  londoncrimes.major_category
Barking and Dagenham  Burglary                       14
                      Criminal Damage                 6
                      Drugs                           1
                      Fraud or Forgery                0
                      Other Notifiable Offences       2
                      Robbery                         2
                      Theft and Handling             49
                      Violence Against the Person    10
Barnet                Burglary                       12
                      Criminal Damage                 2
                      Drugs                          12
                      Fraud or Forgery                0
                      Other Notifiable Offences       0
                      Robbery                         5
                      Theft and Handling             28
                      Violence Against the Person    19
Bexley                Burglary                        

In [63]:
#What are the major crimes. 

In [64]:
df.groupby(['londoncrimes.major_category'])['londoncrimes.value'].sum().head(30).sort_values(ascending=False)

londoncrimes.major_category
Theft and Handling             1003
Violence Against the Person     577
Burglary                        321
Criminal Damage                 200
Drugs                           193
Robbery                          86
Other Notifiable Offences        41
Sexual Offences                   0
Fraud or Forgery                  0
Name: londoncrimes.value, dtype: int64

In [65]:
#running SQL queries directly and examining results.  

In [66]:
df=pd.read_sql('select borough,sum(value) as `Incidents`'
           'from londoncrimes '
           'group by borough order by `Incidents` desc',conn)

In [67]:
df.head()

,borough,incidents
0,Westminster,455028
1,Lambeth,292178
2,Southwark,278809
3,Camden,275147
4,Newham,262024


In [68]:
#let's look at Crimes by year. 

In [69]:
df=pd.read_sql('select * from londoncrimes limit 5000',conn)

In [70]:
df.columns

Index([u'londoncrimes.lsoa_code', u'londoncrimes.borough',
       u'londoncrimes.major_category', u'londoncrimes.minor_category',
       u'londoncrimes.value', u'londoncrimes.year'],
      dtype='object')

In [71]:
df.groupby(['londoncrimes.year'])['londoncrimes.value'].sum().head(20).sort_values(ascending=True)

londoncrimes.year
2010    196
2015    249
2013    264
2012    266
2008    267
2014    268
2016    280
2009    288
2011    343
Name: londoncrimes.value, dtype: int64

In [72]:
#Lets do a simple graph that plots Year against number of crimes. We will be using plotly offline to do this

In [73]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [74]:
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

In [75]:
init_notebook_mode(connected=True)

In [76]:
df.columns

Index([u'londoncrimes.lsoa_code', u'londoncrimes.borough',
       u'londoncrimes.major_category', u'londoncrimes.minor_category',
       u'londoncrimes.value', u'londoncrimes.year'],
      dtype='object')

In [77]:
iplot([Bar(x=df['londoncrimes.year'], y=df['londoncrimes.value'])])

In [78]:
#Lets look at number of crimes by Borough

In [118]:
df=pd.read_sql('select borough, sum(value) as `num_of_crimes`'
              'from londoncrimes '
              'group by borough order by `num_of_crimes` desc',conn )

In [119]:
df.head(20)

,borough,num_of_crimes
0,Westminster,455028
1,Lambeth,292178
2,Southwark,278809
3,Camden,275147
4,Newham,262024
5,Croydon,260294
6,Ealing,251562
7,Islington,230286
8,Tower Hamlets,228613
9,Brent,227551


In [120]:
df.columns

Index([u'borough', u'num_of_crimes'], dtype='object')

In [121]:
iplot([Bar(x=df['borough'],y=df['num_of_crimes'])])

In [122]:
#What time of crimes associate with these boroughs. let's take a look 

In [123]:
#first we need to get the boroughs and create a trace which will assign the major category per borough. 

In [167]:
df=pd.read_sql('select borough, count(*) as `num_of_crimes` from londoncrimes group by borough '
               'order by `num_of_crimes` limit 10 ',conn)
              

In [168]:
boroughs=list(df.borough)

In [169]:
traces = []

In [170]:
for borough in boroughs:
    df = pd.read_sql('select major_category,sum(value) as `num_of_crimes` '
                    'from londoncrimes '
                    'where borough = "{}" '
                    'group by major_category '
                    'order by `num_of_crimes`'.format(borough),conn)
    traces.append(Bar(x=df['major_category'], y=df.num_of_crimes, name=borough.capitalize()))

In [171]:
borough

u'Harrow'

In [174]:
iplot({'data': traces, 'layout': Layout(barmode='stack', xaxis={'tickangle': 30}, margin={'b': 200})})